In [ ]:
%pip install databricks

In [ ]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import compute, jobs
from azure.identity import DefaultAzureCredential

In [ ]:
# TODO: fill these variables with valid values:
uami_client_id = ""
dbw_host = ""

credential = DefaultAzureCredential(managed_identity_client_id=uami_client_id)
token = credential.get_token("2ff814a6-3304-4ab8-85cb-cd0e6f879c1d/.default")


w = WorkspaceClient(host = dbw_host,
                    token = token.token)

In [ ]:
local_script_path = 'ray-job.py'
dbfs_path = "/tmp/ray-job.py"

with open(local_script_path, 'rb') as f:
    w.dbfs.upload(path=dbfs_path, src=f, overwrite=True)
    
cluster_config = compute.ClusterSpec(
    spark_version="15.4.x-cpu-ml-scala2.12",
    node_type_id="Standard_D4ds_v5",
    num_workers=1)

### Create the job 
created_job = w.jobs.create(
    name='ray-job-on-dbw-01',
    tasks=[
        jobs.Task(
            description="Submit Ray Job",
            new_cluster=cluster_config,
            spark_python_task=jobs.SparkPythonTask(
                python_file=f'dbfs:{dbfs_path }',
                parameters=[]
            ),
            task_key="ray_job_task",
            timeout_seconds=0,
        )
    ]
)

print(f"Job submitted successfully. Job ID: {created_job.job_id}")

### Run the job
# run_response = w.jobs.run_now(job_id=created_job.job_id)
run_response = w.jobs.run_now_and_wait(job_id=created_job.job_id)


print(f"Job run triggered successfully. Run ID: {run_response.run_id}")